Try to merge in overloads from other maintainers 
----

- clone to repos 
- new docstubs 
- merge in overloads

In [52]:
# BLANK SLATE
! git -C scratch reset .
! git -C scratch clean . -f

# create docstubs without cleaning up the .rst  docstrings
! stubber docstubs --version 1.23.0 --stub-path scratch --no-autoflake 
# --no-clean-rst

! black scratch
! git -C scratch add .

Removing micropython-v1_23_0-docstubs/
02:11:48 | ℹ️ micropython-stubber 1.23.2a0
02:11:48 | ℹ️ fetch updates
02:11:49 | ℹ️ Switching to v1.23.0
02:11:51 | ℹ️ repos\micropython                        v1.23.0
02:11:51 | ℹ️ repos\micropython-lib                    v1.23.0
02:11:51 | ℹ️ repos\micropython\lib\micropython-lib    v1.23.0
02:11:52 | ℹ️ Get docstubs for MicroPython v1.23.0
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\array\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\asyncio\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\binascii\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\bluetooth\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\btree\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\cmath\__init__.pyi
02:11:52 | ℹ️  - Writing to: 
scratch\micropython-v1_23_0-docstubs\collections\__init__.py

Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\_thread\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\collections\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\deflate\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\errno\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\binascii\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\esp\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\array\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\framebuf\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropyt

#### todo 
- [ ] --no-docstring still copies over the class docstring in array.array
- [x] avoid defining AnyReadableBuf and AnyWritableBuf in multile modules ( _mpy_shed module)
- [x] for now: Include in docstub MODULE_GLUE 
- [ ] check if types for module constants are copied over 
- [ ] Create method to Copy TypeVars and TypeAliasses from source to target ( needed for future merges) 
- [x] manual addition of some classes
     - [ ] ? create a mothod to forcefully copy over a class and all methods while merging 

cmath has some incorrect type definitions for _C 



 copy from C:\Users\josverl\.vscode\extensions\ms-python.vscode-pylance-2024.9.1\dist\typeshed-fallback\stdlib\cmath.pyi


In [53]:
! git -C scratch reset .
# assume root of workspace is the current directory
# # Merge typehints from IntelliJ plugin with docstubs
!stubber enrich --params-only --stubs scratch --docstubs repos\intellij-micropython\typehints



# Merge from Howard Lovatt (MIT) 
# %run -m stubber.stubber enrich --params-only --stubs repos\micropython-stubs\stubs\micropython-v1_23_0-docstubs --source repos\PyBoardTypeshed\micropython_typesheds


02:12:04 | ℹ️ micropython-stubber 1.23.2a0
02:12:04 | ℹ️ Enriching scratch with repos\intellij-micropython\typehints
02:12:04 | ℹ️ Enrich folder scratch.
02:12:04 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\_thread\__init__.pyi 
from repos\intellij-micropython\typehints\stdlib\_thread.pyi
02:12:04 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\array\__init__.pyi from
repos\intellij-micropython\typehints\stdlib\array.pyi
02:12:04 | ℹ️ Change __getitem__ to @overload
02:12:04 | ℹ️ Change __setitem__ to @overload
02:12:04 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\array\__init__.pyi from
repos\intellij-micropython\typehints\stdlib\uarray.pyi
02:12:05 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\asyncio\__init__.pyi 
from repos\intellij-micropython\typehints\stdlib\uasyncio.pyi
02:12:05 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\binascii\__init__.pyi 
from repos\intellij-micropython\typehints\stdlib\binascii.pyi
02:12:05 | ℹ️ Merge scratch\micropython-v1_23_0-docstubs\bin

Replace the `import u<module>` with `import module`

In [54]:
from pathlib import Path


UMOD_TO_UMOD = [
    "from uarray import",
    "from uio import",
    "from usocket import",
    "from uos import",
    "from usys import",
    "from utime import",
]

NO_IMPORT = [
    "from io import AnyReadableBuf",
    "from io import AnyWritableBuf",
]

REPLACE = [
    ("from os import AbstractBlockDev", "from _mpy_shed import AbstractBlockDev"),
]

# Iterate over all files in the directory
directory = Path("scratch/micropython-v1_23_0-docstubs")
for file in directory.glob("**/*"):
    # Check if the file is a regular file
    if file.is_file():
        # Open the file in read mode
        with file.open(mode="r", encoding="utf-8") as f:
            # Read the contents of the file
            lines = f.readlines()

        # Open the file in write mode
        with file.open(mode="w", encoding="utf-8") as f:
            # Iterate over each line in the file
            for line in lines:
                # Check if the line starts with
                if any(line.startswith(l) for l in UMOD_TO_UMOD):
                    # Modify the line to remove the `u` prefix
                    line = "from " + line[6:]
                if any(line.startswith(l) for l in NO_IMPORT):
                    # remove the line
                    line = "# " + line
                for old, new in REPLACE:
                    if line.startswith(old):
                        line = new
                        break

                # Write the modified line to the file
                f.write(line)

In [55]:
HASH_CLASS = '''

class _Hash(ABC):
    """
    Abstract base class for hashing algorithms that defines methods available in all algorithms.
    """

    def update(self, data: AnyReadableBuf, /) -> None:
        """
        Feed more binary data into hash.
        """

    def digest(self) -> bytes:
        """
        Return hash for all data passed through hash, as a bytes object. After this
        method is called, more data cannot be fed into the hash any longer.
        """

    def hexdigest(self) -> str:
        """
        This method is NOT implemented. Use ``binascii.hexlify(hash.digest())``
        to achieve a similar effect.
        """

'''
# append this to scratch\micropython-v1_23_0-docstubs\hashlib\__init__.pyi if it does not exist
with open("scratch/micropython-v1_23_0-docstubs/hashlib/__init__.pyi", "a") as f:
    f.write(HASH_CLASS)

In [56]:
TIMERCHANNEL_CLASS = '''


class TimerChannel(ABC):
    """
    Timer channels are used to generate/capture a signal using a timer.



    TimerChannel objects are created using the Timer.channel() method.
    """

    @abstractmethod
    def callback(self, fun: Callable[[Timer], None] | None, /) -> None:
        """
        Set the function to be called when the timer channel triggers.
        ``fun`` is passed 1 argument, the timer object.
        If ``fun`` is ``None`` then the callback will be disabled.
        """

    @overload
    @abstractmethod
    def capture(self) -> int:
        """
        Get or set the capture value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        capture is the logical name to use when the channel is in input capture mode.
        """

    @overload
    @abstractmethod
    def capture(self, value: int, /) -> None:
        """
        Get or set the capture value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        capture is the logical name to use when the channel is in input capture mode.
        """

    @overload
    @abstractmethod
    def compare(self) -> int:
        """
        Get or set the compare value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        compare is the logical name to use when the channel is in output compare mode.
        """

    @overload
    @abstractmethod
    def compare(self, value: int, /) -> None:
        """
        Get or set the compare value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        compare is the logical name to use when the channel is in output compare mode.
        """

    @overload
    @abstractmethod
    def pulse_width(self) -> int:
        """
        Get or set the pulse width value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        pulse_width is the logical name to use when the channel is in PWM mode.

        In edge aligned mode, a pulse_width of ``period + 1`` corresponds to a duty cycle of 100%
        In center aligned mode, a pulse width of ``period`` corresponds to a duty cycle of 100%
        """

    @overload
    @abstractmethod
    def pulse_width(self, value: int, /) -> None:
        """
        Get or set the pulse width value associated with a channel.
        capture, compare, and pulse_width are all aliases for the same function.
        pulse_width is the logical name to use when the channel is in PWM mode.

        In edge aligned mode, a pulse_width of ``period + 1`` corresponds to a duty cycle of 100%
        In center aligned mode, a pulse width of ``period`` corresponds to a duty cycle of 100%
        """

    @overload
    @abstractmethod
    def pulse_width_percent(self) -> float:
        """
        Get or set the pulse width percentage associated with a channel.  The value
        is a number between 0 and 100 and sets the percentage of the timer period
        for which the pulse is active.  The value can be an integer or
        floating-point number for more accuracy.  For example, a value of 25 gives
        a duty cycle of 25%.
        """

    @overload
    @abstractmethod
    def pulse_width_percent(self, value: int | float, /) -> None:
        """
        Get or set the pulse width percentage associated with a channel.  The value
        is a number between 0 and 100 and sets the percentage of the timer period
        for which the pulse is active.  The value can be an integer or
        floating-point number for more accuracy.  For example, a value of 25 gives
        a duty cycle of 25%.
        """

'''
# append this to scratch\micropython-v1_23_0-docstubs\hashlib\__init__.pyi if it does not exist
# with open("scratch/micropython-v1_23_0-docstubs/pyb/Timer.pyi", "a") as f:
#     f.write(TIMERCHANNEL_CLASS)

In [57]:
MPY_SHED = '''
# Some base types to be used in the stubs

from __future__ import annotations
from _typeshed import PathLike
from abc import abstractmethod
from typing import overload, Protocol, Any
from typing_extensions import TypeVar, TypeAlias
from array import array

AnyReadableBuf : TypeAlias =  bytearray | array | memoryview | bytes
AnyWritableBuf : TypeAlias =  bytearray | array | memoryview

StrOrBytesPath: TypeAlias = str | bytes | PathLike[str] | PathLike[bytes]

_StrOrBytesT = TypeVar("_StrOrBytesT", str, bytes)

class _PathLike(Protocol[_StrOrBytesT]):
    @abstractmethod
    def __fspath__(self) -> _StrOrBytesT:
        """Return the file system path representation of the object, preferably as a `str`."""

_AnyPath: TypeAlias = str | bytes | _PathLike[str] | _PathLike[bytes]
_FdOrAnyPath: TypeAlias = int | _AnyPath

AbstractBlockDev: TypeAlias = Any

'''

from pathlib import Path

target = Path("scratch/micropython-v1_23_0-docstubs/_mpy_shed/__init__.pyi")
target.parent.mkdir(parents=True, exist_ok=True)
with open(target, "w") as f:
    f.write(MPY_SHED)

In [58]:

! ruff check scratch\micropython-v1_23_0-docstubs --fix --unsafe-fixes
# TODO use ruff to sort imports and remove unused imports


Found 1112 errors (1112 fixed, 0 remaining).


In [59]:
# -p defines to root to allow for sibling imports
! pyright scratch\micropython-v1_23_0-docstubs -p scratch\micropython-v1_23_0-docstubs 


Please install the new version or set PYRIGHT_PYTHON_FORCE_VERSION to `latest`

d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\_mpy_shed\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\_mpy_shed\__init__.pyi:18:7 - warning: Type variable "_StrOrBytesT" used in generic protocol "_PathLike" should be covariant (reportInvalidTypeVarUse)
d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\asyncio\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\asyncio\__init__.pyi:412:25 - warning: TypeVar "_T" appears only once in generic function signature
    Use "object" instead (reportInvalidTypeVarUse)
d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\espnow\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-v1_23_0-docstubs\espnow\__init__.pyi:483:35 - warning: "value" is not defined (reportUndefinedVariable)
d:\mypython\micropython-stubber\scratch\micropy